In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6

/content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6


In [3]:
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

In [ ]:
# PPO-based training loop for CodeGenerator
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
from torch import nn
from copy import deepcopy
from tqdm import tqdm
from code_generator_1_3b import CodeGenerator
from code_discriminator import CodeDiscriminator
from requirement_to_code_dataset import get_train_dataloader
from value_head import ValueHead
from bitsandbytes.optim import AdamW8bit
import gc

def batched_discriminator(discriminator, texts, batch_size=1):
    preds = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        logits = discriminator(chunk)
        preds.append(logits.cpu())
    return torch.cat(preds, dim=0).to(discriminator.classifier[0].weight.device)

FORMAT_PENALTY_WEIGHT_H = 1.0 / 1.0
FORMAT_PENALTY_WEIGHT_CPP = 1.0 / 1.0

def compute_format_penalties_h(header_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -1.0 if ("UCLASS" not in text and "USTRUCT" not in text and "UINTERFACE" not in text) else 0.0,
            -1.0 if "#pragma once" not in text else 0.0,
            -1.0 if '#include "CoreMinimal.h"' not in text else 0.0,
            -1.0 if '.generated.h' not in text else 0.0,
            -1.0 if 'GENERATED_BODY()' not in text else 0.0,
            -1.0 if '_API' not in text and ("UCLASS" in text or "UINTERFACE" in text) else 0.0,
        ])
        for text in header_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_H * raw_penalties

def compute_format_penalties_cpp(cpp_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -6.0 if "::" not in text else 0.0,
        ])
        for text in cpp_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_CPP * raw_penalties

def compute_log_ratios(log_probs, ref_log_probs, clip=10):
    log_ratio = (log_probs - ref_log_probs).clamp(min=-clip, max=clip)
    return torch.exp(log_ratio)

def compute_advantages(rewards, values, normalize=False):
    adv = rewards - values.detach()
    if normalize:
        return (adv - adv.mean()) / (adv.std() + 1e-6)
    else:
        return adv

def compute_ppo_loss(ratios, advantages, clip_eps):
    ratios = ratios.squeeze()
    advantages = advantages.squeeze()
    clipped = torch.clamp(ratios, 1 - clip_eps, 1 + clip_eps)

    surrogate1 = ratios * advantages
    surrogate2 = clipped * advantages

    surrogate = torch.where(
        advantages >= 0,
        torch.min(surrogate1, surrogate2),
        torch.max(surrogate1, surrogate2)
    )

    return -torch.mean(surrogate)

def compute_value_loss(values_disc, reward_disc, values_format, reward_format):
    loss_disc = nn.functional.mse_loss(values_disc, reward_disc.to(values_disc.dtype))
    loss_format = nn.functional.mse_loss(values_format, reward_format.to(values_format.dtype))
    return loss_disc + loss_format

def compute_sft_loss(generator, tokenizer, prompts, targets):
    joined_inputs = [p + t for p, t in zip(prompts, targets)]
    inputs = tokenizer(joined_inputs, return_tensors="pt", padding=True, truncation=True,
                       max_length=generator.max_length).to(generator.device)
    labels = inputs["input_ids"].clone()

    prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True,
                              max_length=generator.max_length)
    prompt_lens = [len(p[p != tokenizer.pad_token_id]) for p in prompt_inputs["input_ids"]]
    for i, l in enumerate(prompt_lens):
        labels[i, :l] = -100

    output = generator(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    return output.loss


def update_generator(total_loss, generator, g_optimizer):
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
    g_optimizer.step()
    g_optimizer.zero_grad()

def update_discriminator(discriminator, d_optimizer, reference_texts, generated_texts, device):
    all_texts = reference_texts + generated_texts
    labels = torch.tensor([1] * len(reference_texts) + [0] * len(generated_texts), dtype=torch.float32).to(device)
    preds = batched_discriminator(discriminator, all_texts)
    loss = nn.BCEWithLogitsLoss()(preds, labels)

    d_optimizer.zero_grad()
    loss.backward()
    d_optimizer.step()


def train(generator, train_loader, epochs=3, lr=2e-5, clip_eps=0.2, do_PPO=True):
    generator.train()

    discriminator = CodeDiscriminator(load_path="./checkpoint_1_3b/discriminator").to(device)

    discriminator.train()

    ref_generator = CodeGenerator()

    ref_generator = deepcopy(generator)
    ref_generator.eval()
    for param in ref_generator.parameters():
        param.requires_grad = False

    g_optimizer = AdamW8bit(generator.parameters(), lr=lr)

    d_optimizer = AdamW8bit(discriminator.parameters(), lr=1e-4)

    os.makedirs("./checkpoint_1_3b", exist_ok=True)

    sft_coef = 0.5
    value_coef = 4.0

    do_SFT_h = False
    do_SFT_cpp = False

    if do_PPO == False:
        do_SFT_h = True
        do_SFT_cpp = True

    for epoch in range(epochs):
        loop = tqdm(train_loader)

        for step, batch in enumerate(loop):
            torch.cuda.empty_cache()

            requirements = batch["requirement"]
            reference_headers = batch["header_code"]
            reference_cpps = batch["cpp_code"]

            # === [Phase 1] Header PPO Sampling ===

            header_prompts = [generator.create_short_prompt_header(r) for r in requirements]

            # === [Phase 1-A] SFT Loss & Step ===
            if do_SFT_h:
                sft_loss = compute_sft_loss(generator, generator.tokenizer, header_prompts, reference_headers)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                g_optimizer.zero_grad()
                sft_loss.backward()
                g_optimizer.step()

                # Cleanup SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()

            if do_PPO == True:
                # === [Phase 1-B] PPO Loss ===
                header_output = generator.sample_header_with_partial_grad(requirements, max_track_tokens=512)
                print("\n===== .h =====")
                print(header_output["header_texts"][0])
                print(header_output["header_texts"][1])

                header_texts = header_output["header_texts"]

                with torch.no_grad():
                    ref_header_output = ref_generator.sample_header_with_partial_grad(requirements, max_track_tokens=512)

                with torch.no_grad():
                    d_scores = torch.sigmoid(discriminator(header_texts)).squeeze()
                    reward_disc = 2 * (d_scores - 0.5)
                    format_penalties = compute_format_penalties_h(header_texts).to(generator.device)
                    if format_penalties.mean().item() < -0.01:
                        print("format_penalties < -0.01")
                        do_SFT_h = True
                    else:
                        do_SFT_h = False
                    header_rewards = reward_disc + format_penalties

                log_probs     = header_output["header_log_probs"].mean(dim=1)
                ref_log_probs = ref_header_output["header_log_probs"].mean(dim=1)
                ratios        = compute_log_ratios(log_probs, ref_log_probs)

                values_total, values_disc, values_format = generator.compute_value(header_prompts, header_texts, mode="h")
                advantages = compute_advantages(header_rewards, values_total)

                print(f"reward_disc : {reward_disc}")
                print(f"values_disc : {values_disc}")

                print(f"format_penalties : {format_penalties}")
                print(f"values_format : {values_format}")

                print(f"ratios : {ratios}")
                print(f"advantages : {advantages}")
                ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
                value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
                total_loss = ppo_loss + value_coef * value_loss

                # PPO update
                g_optimizer.zero_grad()
                total_loss.backward()
                g_optimizer.step()

                # === Discriminator training ===
                if reward_disc.mean().item() > -0.4 or step % 10 == 0:
                    print(f"\n[Step {step}] Training header discriminator...")
                    update_discriminator(discriminator, d_optimizer, reference_headers, header_texts, generator.device)

                # === Logging ===
                loop.set_description(f"[Epoch {epoch+1}] Header")
                loop.set_postfix({
                    "ppo":    ppo_loss.item(),
                    "value":  value_loss.item(),
                    "reward": header_rewards.mean().item()
                })

                # === Cleanup ===
                for var_name in [
                    "log_probs", "ref_log_probs", "ratios",
                    "values_total", "values_disc", "values_format",
                    "d_scores", "reward_disc", "format_penalties", "header_rewards",
                    "advantages", "ppo_loss", "value_loss", "total_loss"
                ]:
                    var = locals().get(var_name)
                    if isinstance(var, torch.Tensor):
                        locals()[var_name] = var.detach()
                    del var

                del header_output, ref_header_output, header_texts, header_prompts
                torch.cuda.empty_cache()


            # === [Phase 2] CPP PPO 학습 ===

            cpp_prompts = [generator.create_short_prompt_cpp(r, h) for r, h in zip(requirements, reference_headers)]

            # === [Phase 2-A] SFT Loss & Step ===
            if do_SFT_cpp:
                sft_loss = compute_sft_loss(generator, generator.tokenizer, cpp_prompts, reference_cpps)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                g_optimizer.zero_grad()
                sft_loss.backward()
                g_optimizer.step()

                # Clean up SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()


            if do_PPO == True:
                # === [Phase 2-B] PPO + Value Loss ===

                cpp_output = generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=320)
                print("\n===== .cpp =====")
                print(cpp_output["cpp_texts"][0])
                print(cpp_output["cpp_texts"][1])

                cpp_texts = cpp_output["cpp_texts"]

                with torch.no_grad():
                    ref_cpp_output = ref_generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=320)

                with torch.no_grad():
                    d_scores = torch.sigmoid(discriminator(cpp_texts)).squeeze()
                    reward_disc = 2 * (d_scores - 0.5)
                    format_penalties = compute_format_penalties_cpp(cpp_texts).to(generator.device)
                    if format_penalties.mean().item() < -0.01:
                        print("format_penalties < -0.01")
                        do_SFT_cpp = True
                    else:
                        do_SFT_cpp = False
                    cpp_rewards = reward_disc + format_penalties

                log_probs     = cpp_output["cpp_log_probs"].mean(dim=1)
                ref_log_probs = ref_cpp_output["cpp_log_probs"].mean(dim=1)
                ratios        = compute_log_ratios(log_probs, ref_log_probs)

                values_total, values_disc, values_format = generator.compute_value(cpp_prompts, cpp_texts, mode="cpp")
                advantages = compute_advantages(cpp_rewards, values_total)

                print(f"reward_disc : {reward_disc}")
                print(f"values_disc : {values_disc}")

                print(f"format_penalties : {format_penalties}")
                print(f"values_format : {values_format}")

                print(f"ratios : {ratios}")
                print(f"advantages : {advantages}")
                ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
                value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
                total_loss = ppo_loss + value_coef * value_loss

                # PPO step
                g_optimizer.zero_grad()
                total_loss.backward()
                g_optimizer.step()

                # === Train Discriminator on CPP ===
                if reward_disc.mean().item() > -0.4 or step % 10 == 0:
                    print(f"\n[Step {step}] Training cpp discriminator...")
                    update_discriminator(discriminator, d_optimizer, reference_cpps, cpp_texts, generator.device)

                # === Logging ===
                loop.set_description(f"[Epoch {epoch+1}] CPP")
                loop.set_postfix({
                    "ppo":    ppo_loss.item(),
                    "value":  value_loss.item(),
                    "reward": cpp_rewards.mean().item()
                })

                # === Cleanup ===
                for var_name in [
                    "log_probs", "ref_log_probs", "ratios",
                    "values_total", "values_disc", "values_format",
                    "d_scores", "reward_disc", "format_penalties", "cpp_rewards",
                    "advantages", "ppo_loss", "value_loss", "total_loss"
                ]:
                    var = locals().get(var_name)
                    if isinstance(var, torch.Tensor):
                        locals()[var_name] = var.detach()
                    del var

                del cpp_output, ref_cpp_output, cpp_texts, cpp_prompts

                torch.cuda.empty_cache()


            # === 주기적 ref_generator 동기화 ===
            if step % 10 == 0:
                print(f"\n[Step {step}] Ref generator updated.")
                del ref_generator

                ref_generator = deepcopy(generator)
                ref_generator.eval()
                ref_generator = ref_generator.to(generator.device)
                for p in ref_generator.parameters():
                    p.requires_grad = False

                with torch.no_grad():
                    h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],  max_track_tokens=0)
                    cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"], max_track_tokens=0)

                print("\n===== .h =====")
                print(h_generated_responses["header_texts"][0])
                print("\n===== .cpp =====")
                print(cpp_generated_responses["cpp_texts"][0])

                # Save checkpoint
                generator.save("./checkpoint_1_3b/generator")
                discriminator.save("./checkpoint_1_3b/discriminator")

        with torch.no_grad():
            h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],max_track_tokens=0)
            cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"],max_track_tokens=0)
        print("\n===== .h =====")
        print(h_generated_responses["header_texts"][0])
        print("\n===== .cpp =====")
        print(cpp_generated_responses["cpp_texts"][0])


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_path = "./checkpoint_1_3b"

    generator = CodeGenerator(load_path="./checkpoint_1_3b/generator")

    """
    with torch.no_grad():
        h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],  max_track_tokens=0)
        cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"], max_track_tokens=0)
    print("\n===== .h =====")
    print(h_generated_responses["header_texts"][0])
    print("\n===== .cpp =====")
    print(cpp_generated_responses["cpp_texts"][0])
    """

    train_loader = get_train_dataloader("unreal_code_dataset.jsonl", batch_size=2, shuffle=True, limit= 1160)

    train(generator, train_loader, epochs=5, do_PPO=True)



Loading model + LoRA from ./checkpoint_1_3b/generator


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Freezing all parameters except LoRA blocks...
Trainable params: 1,572,864 / 465,560,576 (0.34%)
Loaded value head.


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading discriminator from ./checkpoint_1_3b/discriminator
Loaded classifier head from ./checkpoint_1_3b/discriminator/classifier.pt
Initializing new model from base: Qwen/Qwen1.5-0.5B
Applying LoRA adaptation...
Freezing all parameters except LoRA blocks...
Trainable params: 1,572,864 / 465,560,576 (0.34%)
No value head to load


...

[Epoch 1] CPP:   6%|▋         | 37/580 [2:37:41<40:41:14, 269.75s/it, ppo=-0.0275, value=0.00878, reward=-0.0572]


===== .h =====
```cpp
#pragma once

#include "CoreMinimal.h"
#include "GameFramework/Actor.h"
#include "PyramidMeshActor.generated.h"

UCLASS()
class YOURPROJECT_API APyramidMeshActor : public AActor
{
	GENERATED_BODY()

public:
	APyramidMeshActor();

protected:
	virtual void BeginPlay() override;

public:
	virtual void Tick(float DeltaTime) override;

private:
	UPROPERTY(EditAnywhere, Category="Pyramid")
	float PyramidHeight;

	UPROPERTY(EditAnywhere, Category="Pyramid")
	float PyramidWidth;

	UPROPERTY(EditAnywhere, Category="Pyramid")
	float PyramidDepth;

	void CreatePyramidMesh();

private:
	void CreatePyramidMeshFromMesh();
};
```
```cpp
#pragma once

#include "CoreMinimal.h"
#include "GameFramework/Actor.h"
#include "SwapMeshActor.generated.h"

UCLASS()
class YOURPROJECT_API ASwapMeshActor : public AActor
{
	GENERATED_BODY()

public:
	ASwapMeshActor();

	UFUNCTION(BlueprintCallable, Category="Mesh")
	void SwapMesh();

protected:
	UPROPERTY(VisibleAnywhere)
	UStaticMesh* Mesh1;


[Epoch 1] Header:   6%|▋         | 37/580 [2:40:26<40:41:14, 269.75s/it, ppo=-0.114, value=0.23, reward=-0.451]     


===== .cpp =====
```cpp
#include "PyramidMeshActor.h"

// Sets default values
APyramidMeshActor::APyramidMeshActor()
{
	// Set this actor to call Tick() every frame.  You can turn this off to improve performance if you don't need it.
	PrimaryActorTick.bCanEverTick = true;

	ProceduralMesh = CreateDefaultSubobject<UProceduralMeshComponent>(TEXT("ProceduralMesh"));
	ProceduralMesh->SetRootComponent(ProceduralMesh);

	CreatePyramidMesh();
}

// Called when the game starts
void APyramidMeshActor::BeginPlay()
{
	Super::BeginPlay();
}

// Called every frame
void APyramidMeshActor::Tick(float DeltaTime)
{
	Super::Tick(DeltaTime);

	CreatePyramidMesh();
}

void APyramidMeshActor::CreatePyramidMesh()
{
	// Create a pyramid mesh
	ProceduralMesh->AddMesh(new UPyramidMesh(0.0f, 0.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 100.0f, 

[Epoch 1] CPP:   7%|▋         | 38/580 [2:44:33<46:59:47, 312.15s/it, ppo=0.965, value=1.96, reward=-1.03]

In [ ]:
generator.save("./checkpoint_1_3b/generator")